In [1]:
import pandas as pd
import itertools
from itertools import product
import re
import sys
from collections import defaultdict, deque
from tqdm import tqdm
import numpy as np
import copy
import csv

In [2]:
ROOT = "../"
DATA  = ROOT + "datasets/"
MODELS = ROOT + "models/"

In [3]:
class CPT:
    def __init__(self, head, parents):
        self.head = head
        self.parents = parents
        self.entries = {}

    def __str__(self):
        comma = ", "
        if len(self.parents) == 0:
            return f"probability ( {self.head.name} ) {{" + "\n" \
                f"  table {comma.join ( map(str,self.entries[tuple()].values () ))};" + "\n" \
                f"}}" + "\n"
        else:
            return f"probability ( {self.head.name} | {comma.join ( [p.name for p in self.parents ] )} ) {{" + "\n" + \
                "\n".join ( [  \
                    f"  ({comma.join(names)}) {comma.join(map(str,values.values ()))};" \
                    for names,values in self.entries.items () \
                ] ) + "\n}\n" 

In [4]:
class Variable:
    def __init__(self, name, values):
        self.name = name
        self.values = values 
        self.cpt = None

    def __str__(self):
        comma = ", "
        return f"variable {self.name} {{" + "\n" \
             + f"  type discrete [ {len(self.values)} ] {{ {(comma.join(self.values))} }};" + "\n" \
             + f"}}" + "\n"

In [5]:
class BayesianNetwork:
    def __init__(self, input_file):
        with open(input_file) as f:
            lines = f.readlines()
        self.variables = {}
        for i in range(len(lines)):
            lines[i] = lines[i].lstrip().rstrip().replace('/', '-')
        i = 0
        while i < len(lines) and not lines[i].startswith("probability"):
            if lines[i].startswith("variable"):
                variable_name = lines[i].rstrip().split(' ')[1]
                i += 1
                variable_def = lines[i].rstrip().split(' ')
                assert(variable_def[1] == 'discrete')
                variable_values = [x for x in variable_def[6:-1]]
                for j in range(len(variable_values)):
                    variable_values[j] = re.sub(r'\(|\)|,', '', variable_values[j])
                variable = Variable(variable_name, variable_values)
                self.variables[variable_name] = variable
            i += 1
        while i < len(lines):
            if lines[i].startswith('probability'):
                split = lines[i].split(' ')
                target_variable_name = split[2]
                variable = self.variables[target_variable_name]
                parents = [self.variables[x.rstrip().lstrip().replace(',', '')] for x in split[4:-2]]
                assert(variable.name == split[2])
                cpt = CPT(variable, parents)
                i += 1
                if len(parents) > 0:
                    nb_lines = 1
                    for p in parents:
                        nb_lines *= len(p.values)
                    for _ in range(nb_lines):
                        cpt_line = [x for x in re.sub(r'\(|\)|,', '', lines[i][:-1]).split()]
                        parent_values = tuple([x for x in cpt_line[:len(parents)]])
                        probabilities = [float(p) for p in cpt_line[len(parents):]]
                        cpt.entries[parent_values] = { v:p for v,p in zip(variable.values,probabilities) }
                        i += 1
                else:
                    cpt_line = [x for x in re.sub(r'\(|\)|,', '', lines[i][:-1]).split()]
                    probabilities = [float(p) for p in cpt_line[1:]]
                    cpt.entries[tuple()] = { v:p for v,p in zip(variable.values,probabilities) }
                variable.cpt = cpt
            i += 1

    def learn_parameters(self, csv_file, laplace_smoothing=True):
        # # Probabilités marginales (si la variable Y n'a pas de parents) :
        # P(Y = y) = count(Y = y) / total_count

        # Probabilités conditionnelles (si Y a des parents X1, X2, ..., Xn) :
        # P(Y = y | X1 = x1, X2 = x2, ..., Xn = xn) = count(Y = y AND X1 = x1 AND ... AND Xn = xn) / count(X1 = x1 AND ... AND Xn = xn)

        # Avec lissage de Laplace (alpha = 1) :
        # P(Y = y | parents) = (count(Y = y, parents) + alpha) / (count(parents) + alpha * K)

        # où :
        # - alpha = 1 (pour le lissage)
        # - K = nombre de valeurs possibles pour Y

        alpha = 1 if laplace_smoothing else 0

        for var in self.variables.values():
            if var.cpt is None:
                var.cpt = CPT(var, [])  # in case CPT is missing

        with open(csv_file, "r") as f:
            reader = csv.DictReader(f)
            data = list(reader)

        for var_name, var in self.variables.items():

            if var.cpt is None:
                continue

            parents = var.cpt.parents
            cpt = {}

            parent_names = []
            for p in parents:
                parent_names.append(p.name)

            parent_value_combinations = []
            parent_values = []
            for p in parent_names:
                parent_values.append(self.variables[p].values)
            
            parent_value_combinations = list(itertools.product(*parent_values))

            for parent_vals in parent_value_combinations:
                parent_vals = tuple(parent_vals)

                counts = {}
                for v in var.values:
                    counts[v] = 0
                total = 0

                # Count occurrences in the data
                for row in data:
                    # Check if this row matches our parent value combination
                    match = True
                    for i in range(len(parent_names)):
                        p = parent_names[i]
                        val = parent_vals[i]
                        if row[p] != val:
                            match = False
                            break
                    
                    # If all parent values match, count this instance
                    if match:
                        y_val = row[var_name]
                        if y_val in counts:
                            counts[y_val] += 1
                            total += 1


                smoothed_total = total + alpha * len(var.values)
                probs = {}
                for v in var.values:
                    probs[v] = (counts[v] + alpha) / smoothed_total
                cpt[parent_vals] = probs

            if not parents:
                counts = {}
                for v in var.values:
                    counts[v] = 0
                total = 0

                for row in data:
                    y_val = row[var_name]
                    if y_val in counts:
                        counts[y_val] += 1
                        total += 1
                smoothed_total = total + alpha * len(var.values)

                probs = {}
                for v in var.values:
                    probs[v] = (counts[v] + alpha) / smoothed_total

                cpt[tuple()] = probs

            var.cpt.entries = cpt


    def write(self,filename):
        with open(filename,"w") as file:
            for var in self.variables.values ():
                file.write(str(var))
            for var in self.variables.values ():
                file.write(str(var.cpt))

    def P_Yisy_given_parents_x(self,Y,y,x=tuple()):
        return self.variables[Y].cpt.entries[x][y]

    def P_Yisy_given_parents(self,Y,y,pa={}):
        x = tuple([ pa[parent.name] for parent in self.variables[Y].cpt.parents ])
        return self.P_Yisy_given_parents_x(Y,y,x)

    def _get_children(self):
        children = defaultdict(list)
        for var in self.variables.values():
            for parent in var.cpt.parents:
                children[parent.name].append(var.name)
        return children

    def _normalize(self, dist):
        total = sum(dist.values())
        return {k: v / total for k, v in dist.items() if total > 0}

    def _get_pi_contribution(self, pname, pval, pi_msgs, evidence):
        if pname in pi_msgs:
            return pi_msgs[pname][pval]
        elif pname in evidence:
            return 1.0 if evidence[pname] == pval else 0.0
        else:
            return 1.0 / len(self.variables[pname].values)

    def _send_messages_to_root(self, node, evidence, children, lambda_msgs):
        if node in lambda_msgs:
            return lambda_msgs[node]

        var = self.variables[node]
        lambda_msg = {val: 1.0 for val in var.values}

        for child in children[node]:
            child_lambda = self._send_messages_to_root(child, evidence, children, lambda_msgs)
            child_var = self.variables[child]
            parent_names = [p.name for p in child_var.cpt.parents]

            new_lambda = {}
            for xi in var.values:
                msg = 0.0
                for xj in child_var.values:
                    def get_vals(pname):
                        if pname == node:
                            return [xi]
                        elif pname in evidence:
                            return [evidence[pname]]
                        else:
                            return self.variables[pname].values

                    all_pa_vals = itertools.product(*[get_vals(pname) for pname in parent_names])

                    for pa in all_pa_vals:
                        pa_dict = dict(zip(parent_names, pa))
                        pa_vals = tuple(pa_dict[p.name] for p in child_var.cpt.parents)
                        prob = child_var.cpt.entries[pa_vals][xj]
                        msg += prob * child_lambda[xj]
                new_lambda[xi] = lambda_msg[xi] * msg
            lambda_msg = new_lambda

        if node in evidence:
            observed = evidence[node]
            lambda_msg = {val: (1.0 if val == observed else 0.0) for val in var.values}

        lambda_msgs[node] = lambda_msg
        return lambda_msg

    def _send_messages_from_root(self, node, pi_msg, evidence, children, lambda_msgs, beliefs, pi_msgs):
        var = self.variables[node]
        lambda_msg = lambda_msgs[node]
        belief = {val: pi_msg[val] * lambda_msg[val] for val in var.values}
        beliefs[node] = self._normalize(belief)
        pi_msgs[node] = pi_msg

        for child in children[node]:
            child_var = self.variables[child]
            parent_names = [p.name for p in child_var.cpt.parents]
            child_pi = {xj: 0.0 for xj in child_var.values}

            for xj in child_var.values:
                total = 0.0
                def get_vals(pname):
                    if pname in evidence:
                        return [evidence[pname]]
                    else:
                        return self.variables[pname].values

                all_pa_vals = itertools.product(*[get_vals(pname) for pname in parent_names])

                for pa in all_pa_vals:
                    pa_dict = dict(zip(parent_names, pa))
                    pa_vals = tuple(pa_dict[p.name] for p in child_var.cpt.parents)
                    prob = child_var.cpt.entries[pa_vals][xj]

                    contrib = 1.0
                    for pname, pval in pa_dict.items():
                        contrib *= self._get_pi_contribution(pname, pval, pi_msgs, evidence)

                    total += contrib * prob

                child_pi[xj] = total

            self._send_messages_from_root(child, child_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)

    def query_marginal(self, query_var, evidence):
        children = self._get_children()
        lambda_msgs = {}
        beliefs = {}
        pi_msgs = {}
        root = next(iter(evidence)) if evidence else query_var
        self._send_messages_to_root(root, evidence, children, lambda_msgs)
        root_var = self.variables[root]
        if root_var.cpt.parents:
            root_pi = {val: 1.0 for val in root_var.values}
        else:
            root_pi = root_var.cpt.entries[tuple()]
        self._send_messages_from_root(root, root_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)
        if query_var not in beliefs:
            self._send_messages_to_root(query_var, evidence, children, lambda_msgs)
            if self.variables[query_var].cpt.parents:
                root_pi = {val: 1.0 for val in self.variables[query_var].values}
            else:
                root_pi = self.variables[query_var].cpt.entries[tuple()]
            self._send_messages_from_root(query_var, root_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)

        return beliefs[query_var]
    
    def query_joint(self, var1, var2, evidence):
        joint_dist = {}
        for val1 in self.variables[var1].values:
            joint_dist[val1] = {}
            for val2 in self.variables[var2].values:
                extended_evidence = dict(evidence)
                extended_evidence[var1] = val1
                extended_evidence[var2] = val2
                marg = self.query_marginal(var2, evidence)
                if var2 not in marg or val2 not in marg:
                    marg = self.query_marginal(var2, evidence)
                if val2 not in marg:
                    continue
                cond = self.query_marginal(var1, {**evidence, var2: val2})
                if var1 not in cond or val1 not in cond:
                    cond = self.query_marginal(var1, {**evidence, var2: val2})
                joint_dist[val1][val2] = cond[val1] * marg[val2]
        return self._normalize_nested(joint_dist)

    def _normalize_nested(self, joint_dist):
        total = sum(v for d in joint_dist.values() for v in d.values())
        if total == 0:
            return joint_dist
        for k1 in joint_dist:
            for k2 in joint_dist[k1]:
                joint_dist[k1][k2] /= total
        return joint_dist

In [ ]:
def _clean_missing_dataframe(df):
    """
    Clean the missing-value dataframe:
    - Replace "?" with NA
    - Convert all non-missing values to stringified integers
    """
    df = df.replace("?", pd.NA)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(int(float(x))) if pd.notna(x) else pd.NA)
    return df


In [ ]:
def evaluate_network(network_path_file, test_dataset_path, miss_dataset_path, verbose=False):
    bn = BayesianNetwork(network_path_file)

    test_df = pd.read_csv(test_dataset_path)
    miss_df = pd.read_csv(miss_dataset_path)
    miss_df = _clean_missing_dataframe(miss_df)

    correct = 0
    total = 0
    confidences = []
    row_logs = [] if verbose else None

    iterator = tqdm(range(len(miss_df)), desc="Evaluating") if verbose else range(len(miss_df))

    for i in iterator:
        evidence = miss_df.iloc[i].dropna().to_dict()
        ground_truth = test_df.iloc[i].to_dict()
        missing_vars = [col for col in miss_df.columns if pd.isna(miss_df.iloc[i][col])]

        for var in missing_vars:
            predicted_dist = bn.query_marginal(var, evidence)
            if not predicted_dist:
                continue

            predicted_val = max(predicted_dist, key=predicted_dist.get)
            confidence = predicted_dist[predicted_val]
            confidences.append(confidence)

            is_correct = (predicted_val == str(ground_truth[var]))
            correct += is_correct
            total += 1
            if verbose and not is_correct:
                row_logs.append(
                    f"[Row {i}] {var} predicted: {predicted_val} (conf: {confidence:.2f}), true: {ground_truth[var]}"
                )


    accuracy = correct / total if total > 0 else 0.0
    avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0

    if verbose:
        print(f"\nMismatches ({len(row_logs)}):")
        print("\n".join(row_logs))
        print(f"\nFinal Accuracy: {accuracy:.4f}")
        print(f"Average Confidence: {avg_confidence:.4f}")

    return accuracy, avg_confidence

In [ ]:
data = "alarm"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 2000/2000 [00:03<00:00, 656.40it/s]


Mismatches (466):
[Row 1] EXPCO2 predicted: 0 (conf: 0.25), true: 2
[Row 2] CO predicted: 0 (conf: 0.89), true: 1
[Row 3] BP predicted: 2 (conf: 0.75), true: 0
[Row 9] HRBP predicted: 0 (conf: 0.33), true: 2
[Row 10] ARTCO2 predicted: 1 (conf: 0.50), true: 2
[Row 15] PRESS predicted: 0 (conf: 0.25), true: 3
[Row 17] EXPCO2 predicted: 0 (conf: 0.25), true: 1
[Row 21] VENTMACH predicted: 0 (conf: 0.33), true: 2
[Row 26] MINVOL predicted: 0 (conf: 0.25), true: 3
[Row 28] PAP predicted: 0 (conf: 0.33), true: 1
[Row 33] EXPCO2 predicted: 0 (conf: 0.25), true: 1
[Row 33] PAP predicted: 0 (conf: 0.33), true: 1
[Row 34] CVP predicted: 2 (conf: 0.70), true: 1
[Row 34] PRESS predicted: 0 (conf: 0.25), true: 3
[Row 40] EXPCO2 predicted: 0 (conf: 0.25), true: 1
[Row 51] FIO2 predicted: 1 (conf: 0.95), true: 0
[Row 52] EXPCO2 predicted: 0 (conf: 0.25), true: 1
[Row 54] HREKG predicted: 0 (conf: 0.33), true: 2
[Row 63] HRBP predicted: 0 (conf: 0.33), true: 2
[Row 64] VENTMACH predicted: 0 (conf: 0.

(0.7671164417791104, 0.7650175380408207)

In [33]:
data = "andes"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 2000/2000 [02:56<00:00, 11.32it/s]


Mismatches (387):
[Row 3] GOAL_130 predicted: 0 (conf: 0.80), true: 1
[Row 8] SNode_26 predicted: 0 (conf: 0.50), true: 1
[Row 16] TRY24 predicted: 0 (conf: 0.50), true: 1
[Row 19] DEFINE23 predicted: 1 (conf: 0.83), true: 0
[Row 20] NEWTONS45 predicted: 0 (conf: 0.50), true: 1
[Row 31] HORIZ53 predicted: 0 (conf: 0.50), true: 1
[Row 37] RApp4 predicted: 0 (conf: 0.50), true: 1
[Row 49] WRITE31 predicted: 0 (conf: 0.50), true: 1
[Row 50] SNode_51 predicted: 1 (conf: 0.83), true: 0
[Row 51] SNode_25 predicted: 0 (conf: 0.50), true: 1
[Row 53] APPLY61 predicted: 0 (conf: 0.50), true: 1
[Row 59] TRY24 predicted: 0 (conf: 0.50), true: 1
[Row 64] VECTOR73 predicted: 0 (conf: 0.50), true: 1
[Row 67] RApp3 predicted: 0 (conf: 0.50), true: 1
[Row 77] SNode_124 predicted: 0 (conf: 0.90), true: 1
[Row 84] DEFINE23 predicted: 0 (conf: 0.50), true: 1
[Row 86] SNode_118 predicted: 0 (conf: 0.80), true: 1
[Row 90] AXIS33 predicted: 1 (conf: 0.80), true: 0
[Row 90] GOAL72 predicted: 0 (conf: 0.50), 

(0.8003095975232198, 0.7825911829030913)

In [34]:
data = "asia"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 2000/2000 [00:00<00:00, 3661.52it/s]


Mismatches (212):
[Row 3] smoke predicted: 1 (conf: 0.66), true: 0
[Row 4] bronc predicted: 1 (conf: 0.80), true: 0
[Row 14] dysp predicted: 0 (conf: 0.80), true: 1
[Row 16] bronc predicted: 0 (conf: 0.89), true: 1
[Row 18] xray predicted: 1 (conf: 0.95), true: 0
[Row 21] smoke predicted: 1 (conf: 0.66), true: 0
[Row 22] smoke predicted: 1 (conf: 0.66), true: 0
[Row 26] smoke predicted: 1 (conf: 0.66), true: 0
[Row 41] dysp predicted: 1 (conf: 0.55), true: 0
[Row 43] smoke predicted: 1 (conf: 0.66), true: 0
[Row 50] smoke predicted: 0 (conf: 0.65), true: 1
[Row 51] smoke predicted: 0 (conf: 0.65), true: 1
[Row 58] dysp predicted: 0 (conf: 0.80), true: 1
[Row 59] dysp predicted: 1 (conf: 0.90), true: 0
[Row 70] dysp predicted: 1 (conf: 0.55), true: 0
[Row 90] bronc predicted: 0 (conf: 0.56), true: 1
[Row 99] bronc predicted: 0 (conf: 0.89), true: 1
[Row 117] dysp predicted: 1 (conf: 0.90), true: 0
[Row 123] smoke predicted: 1 (conf: 0.61), true: 0
[Row 136] dysp predicted: 0 (conf: 0.7

(0.8977820636451301, 0.899873084751848)

In [35]:
data = "sachs"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 2000/2000 [00:00<00:00, 2480.30it/s]


Mismatches (687):
[Row 5] Erk predicted: 2 (conf: 0.59), true: 1
[Row 10] Akt predicted: 0 (conf: 0.82), true: 1
[Row 10] PIP3 predicted: 0 (conf: 0.35), true: 1
[Row 17] Erk predicted: 2 (conf: 0.59), true: 1
[Row 20] Jnk predicted: 0 (conf: 0.33), true: 1
[Row 20] Raf predicted: 1 (conf: 0.65), true: 0
[Row 21] Jnk predicted: 0 (conf: 0.33), true: 1
[Row 24] Erk predicted: 0 (conf: 0.47), true: 1
[Row 26] Raf predicted: 0 (conf: 0.54), true: 2
[Row 29] PIP3 predicted: 0 (conf: 0.35), true: 1
[Row 31] Jnk predicted: 0 (conf: 0.33), true: 1
[Row 34] PIP3 predicted: 0 (conf: 0.35), true: 1
[Row 38] PKC predicted: 1 (conf: 0.44), true: 2
[Row 40] P38 predicted: 0 (conf: 0.33), true: 1
[Row 46] Erk predicted: 2 (conf: 0.59), true: 1
[Row 48] P38 predicted: 0 (conf: 0.33), true: 2
[Row 49] Jnk predicted: 0 (conf: 0.33), true: 1
[Row 49] PKC predicted: 1 (conf: 0.70), true: 0
[Row 50] Mek predicted: 2 (conf: 0.35), true: 0
[Row 53] PIP3 predicted: 0 (conf: 0.39), true: 2
[Row 54] Mek predi

(0.651269035532995, 0.5634955157077822)

In [36]:
data = "sprinkler"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 200/200 [00:00<00:00, 5655.79it/s]


Mismatches (38):
[Row 0] Sprinkler predicted: 1 (conf: 0.52), true: 0
[Row 1] Sprinkler predicted: 1 (conf: 0.52), true: 0
[Row 8] Cloudy predicted: 1 (conf: 0.88), true: 0
[Row 17] Rain predicted: 0 (conf: 0.71), true: 1
[Row 51] Sprinkler predicted: 0 (conf: 0.90), true: 1
[Row 66] Cloudy predicted: 1 (conf: 0.79), true: 0
[Row 88] Sprinkler predicted: 0 (conf: 0.91), true: 1
[Row 92] Sprinkler predicted: 1 (conf: 0.52), true: 0
[Row 95] Cloudy predicted: 0 (conf: 0.56), true: 1
[Row 98] Rain predicted: 1 (conf: 0.52), true: 0
[Row 105] Cloudy predicted: 1 (conf: 0.79), true: 0
[Row 109] Wet_Grass predicted: 1 (conf: 0.90), true: 0
[Row 113] Sprinkler predicted: 0 (conf: 0.50), true: 1
[Row 113] Wet_Grass predicted: 0 (conf: 0.55), true: 1
[Row 117] Cloudy predicted: 0 (conf: 0.71), true: 1
[Row 131] Wet_Grass predicted: 1 (conf: 0.91), true: 0
[Row 134] Wet_Grass predicted: 1 (conf: 0.92), true: 0
[Row 136] Sprinkler predicted: 1 (conf: 0.52), true: 0
[Row 138] Sprinkler predicted:

(0.8155339805825242, 0.8356569612798882)

In [37]:
data = "water"
evaluate_network(
    MODELS + f"{data}/{data}_complete.bif",
    DATA + f"{data}/test.csv",
    DATA + f"{data}/test_missing.csv",
    verbose=True
)

Evaluating: 100%|██████████| 2000/2000 [00:04<00:00, 471.93it/s]


Mismatches (431):
[Row 2] CKNN_12_30 predicted: 0 (conf: 0.35), true: 1
[Row 2] CKNN_12_45 predicted: 0 (conf: 0.33), true: 1
[Row 12] CKNN_12_30 predicted: 0 (conf: 0.80), true: 1
[Row 13] C_NI_12_15 predicted: 1 (conf: 0.45), true: 2
[Row 14] C_NI_12_15 predicted: 1 (conf: 0.67), true: 2
[Row 20] CKNI_12_15 predicted: 1 (conf: 0.39), true: 2
[Row 23] C_NI_12_45 predicted: 2 (conf: 0.50), true: 1
[Row 35] CNOD_12_30 predicted: 0 (conf: 0.61), true: 1
[Row 35] CKNN_12_45 predicted: 0 (conf: 0.33), true: 1
[Row 36] CKNI_12_45 predicted: 0 (conf: 0.33), true: 1
[Row 37] C_NI_12_00 predicted: 1 (conf: 0.40), true: 2
[Row 37] CKNI_12_45 predicted: 0 (conf: 0.33), true: 2
[Row 54] C_NI_12_15 predicted: 3 (conf: 0.37), true: 2
[Row 57] CKNN_12_45 predicted: 0 (conf: 0.33), true: 1
[Row 62] C_NI_12_45 predicted: 3 (conf: 0.60), true: 2
[Row 64] CKNI_12_30 predicted: 1 (conf: 0.40), true: 0
[Row 65] C_NI_12_30 predicted: 3 (conf: 0.48), true: 1
[Row 76] CKNN_12_15 predicted: 0 (conf: 0.80), t

(0.7884143348060874, 0.7783274548616614)